# Notebook 4: Git Authentication Setup & Testing

**What this notebook covers:**
1. Understanding what Git authentication is needed for
2. Creating a test Git repository (the "snapshot repo")
3. Setting up SSH vs HTTPS authentication
4. Configuring your .env file correctly
5. Testing Git operations from Python
6. Troubleshooting common issues

**Prerequisites:**
- Git installed on your system
- A GitHub/GitLab account (free tier is fine)

Let's get started!

## Part 1: Understanding the Snapshot Repository

### What is it?

The Excel Diff Server needs a **separate Git repository** (called the "snapshot repo") to store flattened Excel snapshots.

```
┌─────────────────────┐         ┌─────────────────────┐
│  Your Excel Files   │         │  Snapshot Repo      │
│  (Original Repo)    │────────▶│  (Flattened Text)   │
│                     │         │                     │
│  • data.xlsx        │         │  • data/           │
│  • report.xlsm      │         │    ├─ manifest.json│
│                     │         │    ├─ Sheet1.txt   │
│                     │         │    └─ formulas.txt │
└─────────────────────┘         └─────────────────────┘
```

### When do you need it?

- **Extract endpoint** (`/api/v1/extract`) - Commits snapshots to this repo ✅ NEEDS GIT
- **Flatten endpoint** (`/api/v1/flatten`) - Returns ZIP file ❌ NO GIT NEEDED
- **Compare endpoint** (`/api/v1/compare`) - Compares files ❌ NO GIT NEEDED

**So you only need Git authentication if you're using the Extract endpoint!**

## Part 2: Check Current Git Configuration

Let's see what Git configuration you already have:

In [ ]:
import subprocess
import os
from pathlib import Path

print("=" * 70)
print("GIT CONFIGURATION CHECK")
print("=" * 70)

# Check if git is installed
try:
    result = subprocess.run(['git', '--version'], capture_output=True, text=True, timeout=5)
    print(f"✅ Git installed: {result.stdout.strip()}")
except FileNotFoundError:
    print("❌ Git is not installed!")
    print("   Install: sudo apt-get install git (Linux)")
    print("   Or:      brew install git (Mac)")

print()

# Check git user configuration
try:
    name = subprocess.run(['git', 'config', '--global', 'user.name'], 
                         capture_output=True, text=True, timeout=5).stdout.strip()
    email = subprocess.run(['git', 'config', '--global', 'user.email'], 
                          capture_output=True, text=True, timeout=5).stdout.strip()
    
    if name and email:
        print(f"✅ Git user configured:")
        print(f"   Name:  {name}")
        print(f"   Email: {email}")
    else:
        print("⚠️  Git user not configured")
        print("   Run these commands:")
        print('   git config --global user.name "Your Name"')
        print('   git config --global user.email "your.email@example.com"')
except:
    print("⚠️  Could not check Git configuration")

print()

# Check for SSH keys
ssh_dir = Path.home() / '.ssh'
ssh_keys = []
if ssh_dir.exists():
    for key_file in ['id_rsa', 'id_ed25519', 'id_ecdsa']:
        if (ssh_dir / key_file).exists():
            ssh_keys.append(key_file)

if ssh_keys:
    print(f"✅ SSH keys found: {', '.join(ssh_keys)}")
    print(f"   Location: {ssh_dir}")
else:
    print("⚠️  No SSH keys found")
    print("   SSH keys are needed for git@github.com:... URLs")
    print("   Generate with: ssh-keygen -t ed25519 -C \"your_email@example.com\"")

print("=" * 70)

## Part 3: Create a Test Snapshot Repository

You need to create a repository on GitHub/GitLab to store snapshots.

### Option A: Use GitHub (Recommended)

1. Go to https://github.com/new
2. Repository name: `excel-snapshots-test`
3. Description: "Flattened Excel snapshots for testing"
4. Choose **Private** (recommended) or Public
5. ✅ Initialize with README
6. Click "Create repository"

### Option B: Create Local Repository (For Testing)

If you just want to test locally without GitHub:

In [ ]:
import subprocess
from pathlib import Path
import sys

# Get configuration
project_root = Path.cwd().parent if Path.cwd().name == 'snippets' else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

try:
    from src.core.config import get_settings
    settings = get_settings()
    
    # Use configured path or default
    if settings.SNAPSHOT_REPO_LOCAL_PATH:
        # Get the parent directory from LOCAL_PATH and create a test repo there
        local_path_parent = Path(settings.SNAPSHOT_REPO_LOCAL_PATH).parent
        test_repo_path = local_path_parent / 'excel-snapshots-test'
    else:
        test_repo_path = Path('/tmp') / 'excel-snapshots-test'
        
    print(f"Will create test repository at: {test_repo_path}")
    print(f"(Based on SNAPSHOT_REPO_LOCAL_PATH: {settings.SNAPSHOT_REPO_LOCAL_PATH})\n")
    
except Exception as e:
    print(f"⚠️  Could not load settings: {e}")
    test_repo_path = Path('/tmp') / 'excel-snapshots-test'
    print(f"Using default: {test_repo_path}\n")

if test_repo_path.exists():
    print(f"⚠️  Repository already exists at: {test_repo_path}")
    print("   Skipping creation...")
else:
    print("Creating local test repository...\n")
    
    # Create directory
    test_repo_path.mkdir(parents=True)
    
    # Initialize git repo
    subprocess.run(['git', 'init'], cwd=test_repo_path, check=True)
    subprocess.run(['git', 'config', 'user.name', 'Excel Diff Server'], cwd=test_repo_path, check=True)
    subprocess.run(['git', 'config', 'user.email', 'excel-diff@example.com'], cwd=test_repo_path, check=True)
    
    # Create initial commit
    readme = test_repo_path / 'README.md'
    readme.write_text('# Excel Snapshots Test\n\nThis is a test snapshot repository.')
    subprocess.run(['git', 'add', 'README.md'], cwd=test_repo_path, check=True)
    subprocess.run(['git', 'commit', '-m', 'Initial commit'], cwd=test_repo_path, check=True)
    
    print(f"✅ Created local repository at: {test_repo_path}")
    print(f"\n📝 Add these to your .env file:")
    print(f"   SNAPSHOT_REPO_URL=file://{test_repo_path}")
    
    # Suggest a clone path in the same parent directory
    suggested_clone = test_repo_path.parent / 'snapshot-repo-clone'
    print(f"   SNAPSHOT_REPO_LOCAL_PATH={suggested_clone}")

## Part 4: Authentication Methods

There are three ways to authenticate with Git:

### Method 1: Local File Path (Easiest - No Authentication Needed)

```bash
SNAPSHOT_REPO_URL=file:///workspaces/excel-differ/tmp/excel-snapshots-test
```

**Pros:** No authentication needed, perfect for testing
**Cons:** Only works on your local machine

---

### Method 2: HTTPS with Personal Access Token (Recommended for Work)

```bash
SNAPSHOT_REPO_URL=https://YOUR_TOKEN@github.com/username/excel-snapshots-test.git
```

**Pros:** Easy to set up, works in corporate environments, revocable
**Cons:** Token expires, stored in .env file

**How to get a token:**
1. GitHub: Settings → Developer settings → Personal access tokens → Tokens (classic)
2. Click "Generate new token (classic)"
3. Name: "Excel Diff Server"
4. Expiration: 90 days (or custom)
5. Scopes: ✅ `repo` (full control of private repositories)
6. Generate token
7. **Copy the token** (you won't see it again!)

---

### Method 3: SSH Keys (Best for Personal Projects)

```bash
SNAPSHOT_REPO_URL=git@github.com:username/excel-snapshots-test.git
```

**Pros:** No token expiration, more secure, no password in .env
**Cons:** Harder to set up, may not work in corporate environments

**How to set up SSH:**
1. Generate key: `ssh-keygen -t ed25519 -C "your_email@example.com"`
2. Add to ssh-agent: `eval "$(ssh-agent -s)" && ssh-add ~/.ssh/id_ed25519`
3. Copy public key: `cat ~/.ssh/id_ed25519.pub`
4. GitHub: Settings → SSH and GPG keys → New SSH key
5. Paste your public key
6. Test: `ssh -T git@github.com`

## Part 5: Interactive .env Configuration Helper

Let's configure your .env file step by step:

In [ ]:
from pathlib import Path

# Determine project root
project_root = Path.cwd().parent if Path.cwd().name == 'snippets' else Path.cwd()
env_file = project_root / '.env'

print("=" * 70)
print("INTERACTIVE .ENV CONFIGURATION")
print("=" * 70)
print()
print("Let's configure your Git settings in the .env file.\n")

# Show current configuration
if env_file.exists():
    print("✅ .env file exists\n")
    
    with open(env_file, 'r') as f:
        lines = f.readlines()
    
    # Find Git-related settings
    print("Current Git settings:")
    print("-" * 70)
    
    git_settings = {}
    for line in lines:
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        if '=' in line:
            key, value = line.split('=', 1)
            if any(x in key for x in ['SNAPSHOT_REPO', 'GIT_USER']):
                git_settings[key] = value
                # Hide tokens in display
                if 'URL' in key and '@' in value and value.startswith('https://'):
                    display_value = value.split('@')[0].split('://')[0] + "://***TOKEN***@" + value.split('@')[1]
                    print(f"  {key:30s} = {display_value}")
                else:
                    print(f"  {key:30s} = {value}")
    
    print("-" * 70)
    print()
    
    # Check if configured
    if 'SNAPSHOT_REPO_URL' in git_settings and git_settings['SNAPSHOT_REPO_URL']:
        print("✅ Snapshot repository is configured!")
        repo_url = git_settings['SNAPSHOT_REPO_URL']
        
        if repo_url.startswith('file://'):
            print("   Type: Local file path")
            print("   ℹ️  Good for testing, no authentication needed")
        elif repo_url.startswith('https://'):
            if '@' in repo_url:
                print("   Type: HTTPS with token")
                print("   ℹ️  Token is embedded in URL")
            else:
                print("   Type: HTTPS (no token)")
                print("   ⚠️  This may fail for private repos!")
        elif repo_url.startswith('git@'):
            print("   Type: SSH")
            print("   ℹ️  Requires SSH key configured")
        else:
            print("   Type: Unknown")
            print("   ⚠️  This may not work!")
    else:
        print("⚠️  SNAPSHOT_REPO_URL is not configured")
        print("   The extract endpoint will not work!")
        print("   (But flatten and compare endpoints will work fine)")

else:
    print("❌ .env file does not exist!")
    print(f"   Create it: cp {project_root}/.env.example {env_file}")

print()
print("=" * 70)

## Part 6: Test Git Authentication

Let's test if we can actually clone/access your snapshot repository:

In [ ]:
import sys
from pathlib import Path

# Add project to path
project_root = Path.cwd().parent if Path.cwd().name == 'snippets' else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.core.config import get_settings

settings = get_settings()

print("=" * 70)
print("TESTING GIT AUTHENTICATION")
print("=" * 70)
print()

repo_url = settings.SNAPSHOT_REPO_URL

if not repo_url:
    print("❌ SNAPSHOT_REPO_URL is not set in .env file")
    print("\n📝 To fix:")
    print("   1. Open .env file")
    print("   2. Set SNAPSHOT_REPO_URL to one of:")
    print("      - file:///path/to/your/test-repo (local testing)")
    print("      - https://TOKEN@github.com/user/repo.git (HTTPS)")
    print("      - git@github.com:user/repo.git (SSH)")
    print()
    print(f"   Current SNAPSHOT_REPO_LOCAL_PATH: {settings.SNAPSHOT_REPO_LOCAL_PATH}")
    print("   You can change this to any path you prefer")
else:
    print(f"Repository URL: {repo_url}")
    print(f"Clone will go to: {settings.SNAPSHOT_REPO_LOCAL_PATH}")
    print()
    
    # Try to clone/access the repository
    import subprocess
    import tempfile
    import shutil
    
    test_clone_dir = Path(tempfile.mkdtemp(prefix="git_test_"))
    
    try:
        print("Attempting to clone repository...\n")
        
        # Try git clone
        result = subprocess.run(
            ['git', 'clone', repo_url, str(test_clone_dir)],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode == 0:
            print("✅ SUCCESS! Repository cloned successfully!")
            print(f"   Cloned to: {test_clone_dir}")
            
            # List files
            files = list(test_clone_dir.glob('*'))
            if files:
                print(f"\n   Files in repo: {', '.join([f.name for f in files])}")
            
            print("\n🎉 Git authentication is working!")
            print("   You can use the extract endpoint now.")
            
        else:
            print("❌ FAILED to clone repository\n")
            print("Error output:")
            print(result.stderr)
            print()
            
            # Provide troubleshooting hints
            if 'Authentication failed' in result.stderr or 'authentication' in result.stderr.lower():
                print("💡 TROUBLESHOOTING: Authentication failed")
                print("   - For HTTPS: Check your token is valid and has 'repo' scope")
                print("   - For SSH: Check your SSH key is added to GitHub")
                print("   - Test SSH: ssh -T git@github.com")
            elif 'not found' in result.stderr.lower() or '404' in result.stderr:
                print("💡 TROUBLESHOOTING: Repository not found")
                print("   - Check the repository exists")
                print("   - Check the URL is correct")
                print("   - For private repos, ensure token/SSH key has access")
            elif 'Permission denied' in result.stderr:
                print("💡 TROUBLESHOOTING: Permission denied")
                print("   - For SSH: Your SSH key may not be added to ssh-agent")
                print("   - Run: ssh-add ~/.ssh/id_ed25519")
            else:
                print("💡 TROUBLESHOOTING: Unknown error")
                print("   - Check if the repository URL is correct")
                print("   - Try cloning manually: git clone", repo_url)
    
    except subprocess.TimeoutExpired:
        print("❌ TIMEOUT: Clone took too long (>30 seconds)")
        print("   The repository may be too large or network is slow")
    
    except Exception as e:
        print(f"❌ ERROR: {e}")
    
    finally:
        # Cleanup
        if test_clone_dir.exists():
            shutil.rmtree(test_clone_dir)

print()
print("=" * 70)

## Part 7: Test Python Git Operations

Now let's test Git operations using GitPython (the library the server uses):

In [ ]:
import sys
from pathlib import Path
import tempfile
import shutil

# Add project to path
project_root = Path.cwd().parent if Path.cwd().name == 'snippets' else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.core.config import get_settings

settings = get_settings()

print("=" * 70)
print("TESTING GITPYTHON OPERATIONS")
print("=" * 70)
print()

if not settings.SNAPSHOT_REPO_URL:
    print("❌ SNAPSHOT_REPO_URL not configured")
else:
    try:
        import git
        
        test_dir = Path(tempfile.mkdtemp(prefix="gitpython_test_"))
        print(f"Test directory: {test_dir}\n")
        
        print("Step 1: Cloning repository with GitPython...")
        repo = git.Repo.clone_from(settings.SNAPSHOT_REPO_URL, test_dir)
        print("✅ Clone successful!\n")
        
        print("Step 2: Configuring user...")
        with repo.config_writer() as config:
            config.set_value('user', 'name', settings.GIT_USER_NAME)
            config.set_value('user', 'email', settings.GIT_USER_EMAIL)
        print(f"✅ Configured as: {settings.GIT_USER_NAME} <{settings.GIT_USER_EMAIL}>\n")
        
        print("Step 3: Creating a test file...")
        test_file = test_dir / 'test_from_notebook.txt'
        test_file.write_text('This is a test from the Git authentication notebook!')
        print(f"✅ Created: {test_file.name}\n")
        
        print("Step 4: Adding file to Git...")
        repo.index.add([str(test_file)])
        print("✅ File added to index\n")
        
        print("Step 5: Committing...")
        commit = repo.index.commit('Test commit from Git authentication notebook')
        print(f"✅ Commit created: {commit.hexsha[:7]}\n")
        
        print("Step 6: Pushing to remote...")
        origin = repo.remote('origin')
        push_info = origin.push()
        
        if push_info:
            print("✅ Push successful!\n")
            print("🎉 ALL GIT OPERATIONS WORKING!")
            print("\n   Your snapshot repository is fully configured.")
            print("   The extract endpoint will work correctly.")
        else:
            print("⚠️  Push returned no info (may have succeeded anyway)")
        
        # Cleanup
        shutil.rmtree(test_dir)
        print(f"\n🧹 Cleaned up test directory")
        
    except git.exc.GitCommandError as e:
        print(f"❌ Git command failed: {e}\n")
        
        if 'Authentication failed' in str(e):
            print("💡 Authentication problem detected!")
            print("\nFor HTTPS with token:")
            print("   - Check token is valid")
            print("   - Token should have 'repo' scope")
            print("   - Format: https://TOKEN@github.com/user/repo.git")
            print("\nFor SSH:")
            print("   - Check SSH key is added to GitHub")
            print("   - Test: ssh -T git@github.com")
            print("   - Add key to agent: ssh-add ~/.ssh/id_ed25519")
        
        # Cleanup
        if test_dir.exists():
            shutil.rmtree(test_dir)
    
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        if test_dir.exists():
            shutil.rmtree(test_dir)

print()
print("=" * 70)

## Part 8: Common Configuration Examples

Here are complete .env configurations for different scenarios:

### Example 1: Local Testing (No GitHub)

```bash
# Server Settings
HOST=0.0.0.0
PORT=8000

# Queue Backend
QUEUE_BACKEND=multiprocessing

# Git Settings - Local repository for testing
# Change these paths to wherever you want on your system
SNAPSHOT_REPO_URL=file:///path/to/your/excel-snapshots-test
SNAPSHOT_REPO_LOCAL_PATH=/path/to/your/snapshot-repo-clone
GIT_USER_NAME=Excel Diff Server
GIT_USER_EMAIL=excel-diff@example.com
```

**Setup:**
```bash
# Replace /path/to/your with your actual path
mkdir -p /path/to/your/excel-snapshots-test
cd /path/to/your/excel-snapshots-test
git init
git config user.name "Excel Diff Server"
git config user.email "excel-diff@example.com"
echo "# Excel Snapshots" > README.md
git add README.md
git commit -m "Initial commit"
```

**Examples of paths:**
- Linux/Mac: `/home/username/repos/excel-snapshots-test`
- Codespaces: `/workspaces/excel-differ/tmp/excel-snapshots-test`
- Windows: `C:/Users/username/excel-snapshots-test`

### Example 2: GitHub with HTTPS Token (Work Environment)

```bash
# Server Settings
HOST=0.0.0.0
PORT=8000

# Queue Backend
QUEUE_BACKEND=multiprocessing

# Git Settings - GitHub with personal access token
SNAPSHOT_REPO_URL=https://ghp_yourTokenHere123456789@github.com/yourname/excel-snapshots.git
# Change this to your preferred local directory
SNAPSHOT_REPO_LOCAL_PATH=/path/to/your/snapshot-repo-clone
GIT_USER_NAME=Excel Diff Bot
GIT_USER_EMAIL=excel-diff@yourcompany.com
```

**Setup:**
1. Create repository on GitHub: https://github.com/new
2. Get token: GitHub → Settings → Developer settings → Personal access tokens → Tokens (classic) → Generate new token
3. Select scope: ✅ `repo` (full control)
4. Copy token and use in URL above

**Note:** The `SNAPSHOT_REPO_LOCAL_PATH` is where the server will clone the repository. You can set this to any directory you have write access to.

### Example 3: GitHub with SSH Keys (Personal Projects)

```bash
# Server Settings
HOST=0.0.0.0
PORT=8000

# Queue Backend  
QUEUE_BACKEND=multiprocessing

# Git Settings - GitHub with SSH
SNAPSHOT_REPO_URL=git@github.com:yourname/excel-snapshots.git
# Change this to your preferred local directory
SNAPSHOT_REPO_LOCAL_PATH=/path/to/your/snapshot-repo-clone
GIT_USER_NAME=Your Name
GIT_USER_EMAIL=your.email@example.com
```

**Setup:**
```bash
# 1. Generate SSH key
ssh-keygen -t ed25519 -C "your.email@example.com"

# 2. Start ssh-agent and add key
eval "$(ssh-agent -s)"
ssh-add ~/.ssh/id_ed25519

# 3. Copy public key
cat ~/.ssh/id_ed25519.pub
# Copy the output

# 4. Add to GitHub
# Go to: GitHub → Settings → SSH and GPG keys → New SSH key
# Paste your public key

# 5. Test
ssh -T git@github.com
# Should say: "Hi username! You've successfully authenticated..."
```

**Note:** The `SNAPSHOT_REPO_LOCAL_PATH` can be any directory where you want the server to clone the repository.

### Example 4: No Git (Flatten/Compare Only)

```bash
# Server Settings
HOST=0.0.0.0
PORT=8000

# Queue Backend
QUEUE_BACKEND=multiprocessing

# Git Settings - Leave empty if not using extract endpoint
SNAPSHOT_REPO_URL=
SNAPSHOT_REPO_LOCAL_PATH=/tmp/snapshot-repo
GIT_USER_NAME=Excel Diff Server
GIT_USER_EMAIL=excel-diff@example.com
```

**Note:** The extract endpoint won't work, but flatten and compare will work fine!

The `SNAPSHOT_REPO_LOCAL_PATH` is not used when `SNAPSHOT_REPO_URL` is empty, but it's good to set it anyway in case you enable Git later.

## Part 9: Troubleshooting Guide

### Problem: "Authentication failed"

**For HTTPS with token:**
- ✅ Check token has not expired
- ✅ Token has `repo` scope selected
- ✅ URL format: `https://TOKEN@github.com/user/repo.git`
- ✅ Token is the actual token string (starts with `ghp_` for GitHub)

**For SSH:**
- ✅ SSH key is generated: `ls ~/.ssh/id_ed25519`
- ✅ SSH key is added to ssh-agent: `ssh-add -l`
- ✅ Public key is added to GitHub: Settings → SSH keys
- ✅ Test connection: `ssh -T git@github.com`

---

### Problem: "Repository not found"

- ✅ Repository exists on GitHub
- ✅ URL is correct (check spelling)
- ✅ You have access to the repository
- ✅ For private repos: token/SSH has correct permissions

---

### Problem: "Permission denied (publickey)"

**SSH key not loaded:**
```bash
# Start ssh-agent
eval "$(ssh-agent -s)"

# Add your key
ssh-add ~/.ssh/id_ed25519

# Verify it's loaded
ssh-add -l
```

---

### Problem: "Could not resolve host"

- ✅ Check internet connection
- ✅ Check if `github.com` is accessible: `ping github.com`
- ✅ Corporate firewall may block SSH (port 22)
- ✅ Try HTTPS instead of SSH

---

### Problem: Works manually but not from Python

**SSH keys not available to Python process:**
```bash
# Make sure ssh-agent is running
eval "$(ssh-agent -s)"
ssh-add ~/.ssh/id_ed25519

# Start server in the same shell session
python -m src.api.main
```

**Or use HTTPS with token** (more reliable for server processes)

## Part 10: Quick Setup Command

Run this to generate a ready-to-use .env configuration based on your choice:

In [ ]:
from pathlib import Path
import sys

project_root = Path.cwd().parent if Path.cwd().name == 'snippets' else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

env_file = project_root / '.env'

print("=" * 70)
print("QUICK .ENV SETUP HELPER")
print("=" * 70)
print()

# Get current SNAPSHOT_REPO_LOCAL_PATH if .env exists
current_local_path = "/tmp/snapshot-repo"
try:
    from src.core.config import get_settings
    settings = get_settings()
    if settings.SNAPSHOT_REPO_LOCAL_PATH:
        current_local_path = str(settings.SNAPSHOT_REPO_LOCAL_PATH)
except:
    pass

print(f"Current SNAPSHOT_REPO_LOCAL_PATH: {current_local_path}")
print("(You can change this to any directory you prefer)\n")

print("Choose your Git setup:")
print()
print("1. Local file (testing only, no GitHub)")
print("2. GitHub HTTPS with token (recommended for work)")
print("3. GitHub SSH (recommended for personal)")
print("4. No Git (flatten/compare only)")
print()

choice = input("Enter choice (1-4): ").strip()

config_lines = []

if choice == '1':
    print("\n📝 Configuration for LOCAL FILE")
    local_path = input(f"Local clone path [{current_local_path}]: ").strip() or current_local_path
    
    # Suggest test repo in same parent directory
    parent_dir = Path(local_path).parent
    test_repo = parent_dir / 'excel-snapshots-test'
    
    config_lines = [
        f"SNAPSHOT_REPO_URL=file://{test_repo}",
        f"SNAPSHOT_REPO_LOCAL_PATH={local_path}",
        "GIT_USER_NAME=Excel Diff Server",
        "GIT_USER_EMAIL=excel-diff@example.com",
    ]
    print("\n⚠️  Don't forget to create the test repository:")
    print(f"   mkdir -p {test_repo}")
    print(f"   cd {test_repo} && git init && git config user.name 'Excel Diff Server'")
    print(f"   echo '# Excel Snapshots' > README.md && git add README.md && git commit -m 'Initial'")
    print("\n   Or run the 'Part 3' cell above to create it.")

elif choice == '2':
    print("\n📝 Configuration for GITHUB HTTPS")
    username = input("GitHub username: ").strip()
    repo_name = input("Repository name: ").strip()
    token = input("Personal access token (starts with ghp_): ").strip()
    user_name = input("Your name for commits: ").strip()
    user_email = input("Your email for commits: ").strip()
    local_path = input(f"Local clone path [{current_local_path}]: ").strip() or current_local_path
    
    config_lines = [
        f"SNAPSHOT_REPO_URL=https://{token}@github.com/{username}/{repo_name}.git",
        f"SNAPSHOT_REPO_LOCAL_PATH={local_path}",
        f"GIT_USER_NAME={user_name}",
        f"GIT_USER_EMAIL={user_email}",
    ]
    print("\n⚠️  Make sure:")
    print(f"   - Repository exists: https://github.com/{username}/{repo_name}")
    print("   - Token has 'repo' scope")

elif choice == '3':
    print("\n📝 Configuration for GITHUB SSH")
    username = input("GitHub username: ").strip()
    repo_name = input("Repository name: ").strip()
    user_name = input("Your name for commits: ").strip()
    user_email = input("Your email for commits: ").strip()
    local_path = input(f"Local clone path [{current_local_path}]: ").strip() or current_local_path
    
    config_lines = [
        f"SNAPSHOT_REPO_URL=git@github.com:{username}/{repo_name}.git",
        f"SNAPSHOT_REPO_LOCAL_PATH={local_path}",
        f"GIT_USER_NAME={user_name}",
        f"GIT_USER_EMAIL={user_email}",
    ]
    print("\n⚠️  Make sure:")
    print(f"   - Repository exists: https://github.com/{username}/{repo_name}")
    print("   - SSH key is added to GitHub")
    print("   - Test: ssh -T git@github.com")

elif choice == '4':
    print("\n📝 Configuration for NO GIT")
    local_path = input(f"Local clone path [{current_local_path}]: ").strip() or current_local_path
    
    config_lines = [
        "SNAPSHOT_REPO_URL=",
        f"SNAPSHOT_REPO_LOCAL_PATH={local_path}",
        "GIT_USER_NAME=Excel Diff Server",
        "GIT_USER_EMAIL=excel-diff@example.com",
    ]
    print("\n⚠️  Note: Extract endpoint will NOT work")
    print("   But flatten and compare will work fine!")

else:
    print("\n❌ Invalid choice")
    config_lines = None

if config_lines:
    print("\n" + "=" * 70)
    print("Add these lines to your .env file:")
    print("=" * 70)
    for line in config_lines:
        print(line)
    print("=" * 70)
    
    if env_file.exists():
        update = input("\nUpdate .env file now? (y/n): ").strip().lower()
        if update == 'y':
            # Read existing file
            with open(env_file, 'r') as f:
                lines = f.readlines()
            
            # Update Git-related lines
            updated_lines = []
            keys_to_update = {line.split('=')[0] for line in config_lines if '=' in line}
            updated_keys = set()
            
            for line in lines:
                if '=' in line and not line.strip().startswith('#'):
                    key = line.split('=')[0].strip()
                    if key in keys_to_update:
                        # Replace this line
                        new_value = next(l for l in config_lines if l.startswith(key))
                        updated_lines.append(new_value + '\n')
                        updated_keys.add(key)
                    else:
                        updated_lines.append(line)
                else:
                    updated_lines.append(line)
            
            # Add any keys that weren't in the file
            for config_line in config_lines:
                key = config_line.split('=')[0]
                if key not in updated_keys:
                    updated_lines.append(config_line + '\n')
            
            # Write back
            with open(env_file, 'w') as f:
                f.writelines(updated_lines)
            
            print("\n✅ .env file updated!")
            print("\n🔄 Restart your API server to load the new configuration.")
    else:
        print("\n⚠️  .env file doesn't exist yet")
        print(f"   Create it: cp {project_root}/.env.example {env_file}")

## ✅ Setup Complete!

**What you learned:**
1. ✅ What the snapshot repository is and why you need it
2. ✅ Three authentication methods (local, HTTPS, SSH)
3. ✅ How to configure .env file for Git
4. ✅ How to test Git authentication
5. ✅ How to troubleshoot common issues

**Next Steps:**

1. **If Git is working**: Try the extract endpoint in notebook 03!
2. **If Git is NOT configured**: You can still use flatten and compare endpoints
3. **To test everything**: Go to [03_test_api.ipynb](03_test_api.ipynb)

**Quick Reference:**

| Endpoint | Needs Git? | What it does |
|----------|-----------|-------------|
| `/api/v1/extract` | ✅ YES | Flatten + commit to repo |
| `/api/v1/flatten` | ❌ NO | Flatten + return ZIP |
| `/api/v1/compare` | ❌ NO | Compare two files |

**Need Help?**
- Read [docs/GIT_AUTHENTICATION.md](../docs/GIT_AUTHENTICATION.md)
- Check [docs/ARCHITECTURE.md](../docs/ARCHITECTURE.md)
- Re-run the test cells in this notebook

**Happy coding!** 🚀